<a href="https://colab.research.google.com/github/sunny0103/DeepLearning_nlp_projects/blob/main/AIconnect_MRC/AIconnect_MRC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers[torch] datasets evaluate

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
cd "/content/drive/MyDrive/Data/aiconnect_doc_mrc"

/content/drive/MyDrive/Data/aiconnect_doc_mrc


In [4]:
import pandas as pd
import numpy as np
import json

import os
import random
from tqdm import tqdm

import torch
from transformers import (AutoTokenizer,
                          DefaultDataCollator,
                          AutoModelForQuestionAnswering,
                          TrainingArguments,
                          Trainer)

from datasets import load_dataset
import evaluate
import collections

In [5]:
def seed_everything(seed):
  random.seed(seed)
  os.environ['PYTHONHASHSEED'] = str(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True
  torch.backends.cudnn.benchmark = True

seed_everything(42)

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:32"

In [6]:
# data = []
# with open('./train.json', encoding="utf-8") as f:
#     squad = json.load(f)
#     for group in squad['data']:
#       for passage in group['paragraphs']:
#           context = passage['context']
#           for qa in passage['qas']:
#               question = qa['question']
#               qu_id = qa['question_id']
#               for ans in qa['answers']:
#                   if qa['is_impossible']:
#                     text = ''
#                     answer_start = -1
#                   else:
#                     text = ans['text']
#                     answer_start = ans['answer_start']


#                     data.append({
#                       'id':qu_id,
#                       'context':context,
#                       'question':question,
#                       'answers':{
#                           'text':[text],
#                           'answer_start': [answer_start]
#                       }
#                   })

# with open("custom_data.json", "w") as jsonFile:
#     json.dump(data, jsonFile)

In [7]:
dataset = load_dataset('json', data_files='custom_data.json', split='train')
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['question', 'id', 'context', 'answers'],
    num_rows: 28024
})

In [8]:
squad = dataset.train_test_split(.2)
squad

DatasetDict({
    train: Dataset({
        features: ['question', 'id', 'context', 'answers'],
        num_rows: 22419
    })
    test: Dataset({
        features: ['question', 'id', 'context', 'answers'],
        num_rows: 5605
    })
})

In [9]:
squad["train"][0]

{'question': '더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니',
 'id': 'QUES_UniBPiuRnf',
 'context': '경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).',
 'answers': {'answer_start': [84], 'text': ['대한민국']}}

In [10]:
print("Context: ", squad["train"][0]["context"])
print("Question: ", squad["train"][0]["question"])
print("Answer: ", squad["train"][0]["answers"])

Context:  경제협력개발기구(OECD)가 발표한 ‘더 나은 삶 지수(Better Life Index, BLI)’ 중 하나인 ‘공동체(Community)’ 지수에서 대한민국은 러시아와 브라질을 포함한 36개 국가 중 최하위를 기록했다. 한국 사회에서 공동체가 파괴되어온 주요인으로 다양한 의견이 개진되었으나, 한국도시의 오래된 특징인 아파트와 고립된 생활 공간, 차도와 인도의 구분이 없는 길과 건물 배치가 이웃 간 소통을 가로막고 갈등이 생겨도 이를 원활하게 해결할 수 없는 환경을 조성했다는 지적이 주목을 받았다(김재형, 2016). 도움이 필요할 때 이웃의 관심과 손길을 기대하기 어려운 환경은 고독사, 아동학대, 가정폭력, 자살 등 다양한 사회문제를 방치하고 악화시키는 데 일조한다. 이러한 공동체 와해와 지역현안 문제를 주민들의 참여를 통해 적극적으로 해결하고자 시행된 정책사업 중 하나가 2010년부터 각 지자체에 본격적으로 도입된 마을 만들기 사업이다. 마을 만들기 사업의 유형은 주체와 내용에 따라 다양한 형태로 발현되지만, 그 핵심은 주민참여를 주요 동력으로 공동의 지역문제와 욕구를 해결하고, 공동체 회복을 통해 주민의 삶의 질을 향상시킨다는 정책적 목적으로 귀결된다(박선희, 2014).
Question:  더 나은 삶 지수 중 하나인 공동체 지수에서 어느 나라가 최하위를 기록했니
Answer:  {'answer_start': [84], 'text': ['대한민국']}


In [11]:
MODEL_NAME ='monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/241k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/492k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/169 [00:00<?, ?B/s]

In [12]:
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=1024,
        truncation="only_second",
        return_offsets_mapping=True,
        padding="max_length",
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = answer["answer_start"][0] + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label it (0, 0)
        if offset[context_start][0] > end_char or offset[context_end][1] < start_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end token positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [13]:
tokenized_squad = squad.map(preprocess_function, batched=True, remove_columns=squad["train"].column_names)

Map:   0%|          | 0/22419 [00:00<?, ? examples/s]

Map:   0%|          | 0/5605 [00:00<?, ? examples/s]

In [14]:
data_collator = DefaultDataCollator()

In [15]:
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

config.json:   0%|          | 0.00/870 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/458M [00:00<?, ?B/s]

Some weights of BigBirdForQuestionAnswering were not initialized from the model checkpoint at monologg/kobigbird-bert-base and are newly initialized: ['qa_classifier.output.dense.weight', 'qa_classifier.intermediate.dense.weight', 'qa_classifier.output.LayerNorm.weight', 'qa_classifier.intermediate.dense.bias', 'qa_classifier.output.dense.bias', 'qa_classifier.qa_outputs.bias', 'qa_classifier.output.LayerNorm.bias', 'qa_classifier.qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
BATCH_SIZE = 16
NUM_EPOCHS = 3

training_args = TrainingArguments(
    output_dir="aiconnect_mrc",
    evaluation_strategy="epoch",
    learning_rate= 2e-5,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    weight_decay=0.01,
    gradient_accumulation_steps = 8,
    fp16=True,
    gradient_checkpointing=True

)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_squad["train"],
    eval_dataset=tokenized_squad["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,0.585250
1,No log,0.527208
2,0.811500,0.538571


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


TrainOutput(global_step=525, training_loss=0.7924826322283064, metrics={'train_runtime': 5988.5488, 'train_samples_per_second': 11.231, 'train_steps_per_second': 0.088, 'total_flos': 3.705435883956634e+16, 'train_loss': 0.7924826322283064, 'epoch': 3.0})

In [18]:
# data = []
# with open('./test.json', encoding="utf-8") as f:
#     squad = json.load(f)
#     for group in squad['data']:
#       for passage in group['paragraphs']:
#           context = passage['context']
#           for qa in passage['qas']:
#               question = qa['question']
#               qu_id = qa['question_id']
#               for ans in qa['answers']:
#                   if qa['is_impossible']:
#                     text = ''
#                     answer_start = -1
#                   else:
#                     text = ans['text']
#                     answer_start = ans['answer_start']


#                     data.append({
#                       'id':qu_id,
#                       'context':context,
#                       'question':question,
#                       'answers':{
#                           'text':[text],
#                           'answer_start': [answer_start]
#                       }
#                   })

# with open("custom_data.json", "w") as jsonFile:
#     json.dump(data, jsonFile)